# Group Project #4

For this assignment, we will be using crime data from LA times to perform a spatial autocorrelation analysis. 

We will begin by importing the libraries we will be using.

In [ ]:
# to read and wrangle data
import pandas as pd

# to import data from LA Data portal
from sodapy import Socrata

# to create spatial data
import geopandas as gpd

# for basemaps
import contextily as ctx

# For spatial statistics
import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation

import libpysal as lps

# Graphics
import matplotlib.pyplot as plt
import plotly.express as px

Next, we will bring in a census geography that will allow us to summarize the location of crimes committed in Los Angeles.

In [ ]:
# read downloaded geojson file from census reporter
gdf = gpd.read_file('data/acs2018_5yr_B01003_15000US060372711003.geojson')

In [ ]:
gdf.info()

In [ ]:
# trim the data to the bare minimum columns
gdf = gdf[['geoid','B01003001','geometry']]

# rename the columns
gdf.columns = ['FIPS','TotalPop','geometry']

In [ ]:
# last rows
gdf.tail()

In [ ]:
# We want to delete last column which is for the entire city of LA
gdf=gdf.drop(2515)

In [ ]:
# fix FIPS code
gdf['FIPS'] = gdf['FIPS'].str.replace('15000US','')
gdf.tail()

In [ ]:
# sort by total pop
gdf.sort_values(by='TotalPop').head(20)

In [ ]:
# delete zero population geographies
gdf = gdf[gdf['TotalPop']>100]

Now, let's move on to mapping the census block groups.

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
gdf = gdf.to_crs(epsg=3857)

In [ ]:
# Now, we want to plot this information
ax=gdf.plot(figsize=(12,12),
                      color='grey', 
                      edgecolor='white',
                      alpha=0.5)
# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Now, we want to import crime data for Los Angeles from lacity.org. We will be using the timeline of March to April 2020 because the data is so dense.

In [ ]:
# Next, we will connect to the data portal
client = Socrata("data.lacity.org", None)

results = client.get("2nrs-mtv8", 
                     limit=30000,
                     where = "date_rptd between '2020-03-01T00:00:00' and '2020-04-30T00:00:00'",
                     order='crm_cd_desc')

# Convert to pandas DataFrame
crimes = pd.DataFrame.from_records(results)

In [ ]:
crimes.shape

In [ ]:
# convert pandas dataframe to geodataframe
crimes = gpd.GeoDataFrame(crimes, 
                         crs='EPSG:4326',
                         geometry=gpd.points_from_xy(crimes.lon, crimes.lat))

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
crimes = crimes.to_crs(epsg=3857)

In [ ]:
# convert lat/lon to floats
crimes.lon = crimes.lon.astype('float')
crimes.lat = crimes.lat.astype('float')

In [ ]:
# Let's map it
ax = crimes.plot(figsize=(12,12),
                  color='red',
                  markersize=1)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
# Now we want to fix the error we se above. We want to subset the zero coordinate records
crimes[crimes.lon==0]

In [ ]:
# drop the unmapped rows
crimes.drop(crimes[crimes.lon==0].index,inplace=True)

In [ ]:
# Let's map it again and see what it looks like
ax = crimes.plot(figsize=(12,12),
                  color='blue',
                  markersize=1,
                  alpha=0.5)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Now, we want to combine both the layers we have created.

In [ ]:
# get the bounding box coordinates for the arrest data
minx, miny, maxx, maxy = crimes.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

In [ ]:
# set up the plot canvas with plt.subplots
fig, ax = plt.subplots(figsize=(15, 15))

# block groups
gdf.plot(ax=ax, # this puts it in the ax plot
        color='gray', 
        edgecolor='white',
        alpha=0.5)

# arrests
crimes.plot(ax=ax, # this also puts it in the same ax plot
            color='blue',
            markersize=1,
            alpha=0.2)

# use the bounding box coordinates to set the x and y limits
ax.set_xlim(minx - 1000, maxx + 1000) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - 1000, maxy + 1000)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Next, we want to perform a spatial join.

In [ ]:
join = gpd.sjoin(gdf, crimes, how='right')
join.head()

In [ ]:
crimes_by_gdf = join.FIPS.value_counts().rename_axis('FIPS').reset_index(name='crimes_count')

In [ ]:
crimes_by_gdf.head()

In [ ]:
# make a bar chart of top 20 geographies
crimes_by_gdf[:20].plot.bar(figsize=(20,4),
                             x='FIPS',
                             y='crimes_count')

Now, we want to join the summary table back to the GeoDataFrame.

In [ ]:
gdf=gdf.merge(crimes_by_gdf,on='FIPS')

In [ ]:
#Now, we have a count column.
gdf.head()

Now, let's look at crimes committed in Los Angeles per 1000 people.

In [ ]:
gdf['crimes_per_1000'] = gdf['crimes_count']/gdf['TotalPop']*1000

In [ ]:
gdf.sort_values(by="crimes_per_1000").tail()

In [ ]:
# map the top 20 geographies
ax = gdf.sort_values(by='crimes_per_1000',ascending=False)[:20].plot(figsize=(12,10),
                                                             color='blue',
                                                             edgecolor='white',
                                                             alpha=0.5,legend=True)


# title
ax.set_title('Top 20 locations of Crimes Committed in Los Angeles per 1000 people (March-April 2020)')

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
#Let's make a choropleth map
ax = gdf.plot(figsize=(15,15),
                        column='crimes_per_1000',
                        legend=True,
                        alpha=0.8,
                        cmap='Blues',
                        scheme='quantiles')

ax.axis('off')
ax.set_title('March to April 2020 crimes committed in LA per 1000 people',fontsize=22)
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Now, we want to conduct spatial autocrrelation to determine what degree an existing pattern could potentially be random.

We will be using Moran's I statistic.

In [ ]:
# calculate spatial weight
wq =  lps.weights.KNN.from_dataframe(gdf,k=8)
wq.transform = 'r'

In [ ]:
# create a new column for the spatial lag
gdf['crimes_per_1000_lag'] = lps.weights.lag_spatial(wq, gdf['crimes_per_1000'])

In [ ]:
gdf.sort_values(by='crimes_per_1000',ascending=False).sample(10)

Now, let's consider the donut and diamond geographies to better understand the significance of the spatial lag values.

In [ ]:
gdf[gdf['FIPS'].isin(['060372739022', '060379800241'])]

In [ ]:
# set the mapbox access token
token = 'pk.eyJ1IjoiZGhleWRhciIsImEiOiJja2llM3k3bHExYTN5Mnlueng0ZTd6bGQ2In0.TIg8HNU19SXwuGSdJSkRtQ'
px.set_mapbox_access_token(token)

In [ ]:
# subset donut, project to WGS84, and get its centroid
gdf_donut = gdf[gdf.FIPS=='060372739022']
gdf_donut = gdf_donut.to_crs('epsg:4326')

# what's the centroid?
minx, miny, maxx, maxy = gdf_donut.geometry.total_bounds
center_lat_donut = (maxy-miny)/2+miny
center_lon_donut = (maxx-minx)/2+minx

In [ ]:
# subset diamond, project to WGS84, and get its centroid
gdf_diamond = gdf[gdf.FIPS=='060379800241']
gdf_diamond = gdf_diamond.to_crs('epsg:4326')

# what's the centroid?
minx, miny, maxx, maxy = gdf_diamond.geometry.total_bounds
center_lat_diamond = (maxy-miny)/2+miny
center_lon_diamond = (maxx-minx)/2+minx

In [ ]:
px.choropleth_mapbox(gdf_donut, 
                     geojson=gdf_donut.geometry, 
                     locations=gdf_donut.index, 
                     mapbox_style="satellite-streets",
                     zoom=14, 
                     center = {"lat": center_lat_donut, "lon": center_lon_donut},
                     hover_data=['crimes_count','crimes_per_1000','crimes_per_1000_lag'],
                     opacity=0.4,
                     title='The Donut')

In [ ]:
px.choropleth_mapbox(gdf_diamond, 
                     geojson=gdf_diamond.geometry, 
                     locations=gdf_diamond.index, 
                     mapbox_style="satellite-streets",
                     zoom=12, 
                     center = {"lat": center_lat_diamond, "lon": center_lon_diamond},
                     hover_data=['crimes_count','crimes_per_1000','crimes_per_1000_lag'],
                     opacity=0.4,
                     title='The Diamond')

Now, we want to map the entire dataframe with the new spatial lag column we created.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

# spatial lag choropleth
gdf.plot(ax=ax,
         figsize=(15,15),
         column='crimes_per_1000_lag',
         legend=True,
         alpha=0.8,
         cmap='Blues',
         scheme='quantiles')

# uncomment this to see the actual point locations of arrests
# arrests.plot(ax=ax, 
#              color='blue',
#              markersize =1,
#              alpha=0.2, 
#              legend=True)

ax.axis('off')
ax.set_title('March-April 2020 Crimes Committed in Los Angeles per 1000 people',fontsize=22)

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Let's compare the two maps we've made.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

ax1, ax2 = axs

# regular count map on the left
gdf.plot(column='crimes_per_1000', 
            cmap='Blues', 
            scheme='quantiles',
            k=5, 
            edgecolor='white', 
            linewidth=0., 
            alpha=0.75, 
            ax=ax1 
           )


ax1.axis("off")
ax1.set_title("Crimes Committed in Los Angeles per 1000")

# spatial lag map on the right
gdf.plot(column='crimes_per_1000_lag', 
            cmap='Blues', 
            scheme='quantiles',
            k=5, 
            edgecolor='white', 
            linewidth=0., 
            alpha=0.75, 
            ax=ax2 
           )

ax2.axis("off")
ax2.set_title("Crimes Committed in Los Angeles per 1000 - Spatial Lag")

plt.show()

Next, we want to create an interactive spatial lag satellite map.

In [ ]:
# interactive version needs to be in WGS84
gdf_web = gdf.to_crs('EPSG:4326')

In [ ]:
# what's the centroid?
minx, miny, maxx, maxy = gdf_web.geometry.total_bounds
center_lat_gdf_web = (maxy-miny)/2+miny
center_lon_gdf_web = (maxx-minx)/2+minx

In [ ]:
# the median?
median = gdf_web.crimes_per_1000_lag.median()

In [ ]:
fig = px.choropleth_mapbox(gdf_web, 
                     geojson=gdf_web.geometry, 
                     locations=gdf_web.index, 
                     mapbox_style="satellite-streets",
                     zoom=9, 
                     color='crimes_per_1000_lag',
                     color_continuous_scale='Blues',
                     color_continuous_midpoint =median,
                     range_color =(0,median*2),
                     hover_data=['crimes_count','crimes_per_1000','crimes_per_1000_lag'],
                     center = {"lat": center_lat_gdf_web, "lon": center_lon_gdf_web},
                     opacity=0.8,
                     width=1200,
                     height=800,
                     labels={
                             'crimes_per_1000_lag':'Crimes Committed in Los Angeles per 1000 (Spatial Lag)',
                             'crimes_per_1000':'Crimes Committed in Los Angeles per 1000',
                     })
fig.update_traces(marker_line_width=0.1, marker_line_color='white')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

Let's move towards the Moran's Plot.

In [ ]:
y = gdf.crimes_per_1000
moran = Moran(y, wq)
moran.I

We have a positive spatial autocorrelation, which means high values are close to high values, or low values are close to low values.

In [ ]:
fig, ax = moran_scatterplot(moran, aspect_equal=True)
plt.show()

In [ ]:
plot_moran_simulation(moran,aspect_equal=False)

In [ ]:
moran.p_sim

We can reject the hypothesis that the map is random becuase the p-value is small.

Now, we want Local Indicators of Spatial Association (LISA) to calculate the clusters by classifying it into these 4 groups: High values near to high values (HH), low values with nearby low values (LL), low values with high values (LH) and high values with nearbly low values (HL)

In [ ]:
# calculate local moran values
lisa = esda.moran.Moran_Local(y, wq)

In [ ]:
# Plot
fig, ax = moran_scatterplot(lisa, p=0.05)
ax.set_xlabel("Crimes Committed")
ax.set_ylabel('Spatial Lag of Crimes Committed')
plt.text(1.95, 0.5, "HH", fontsize=25)
plt.text(1.95, -1.5, "HL", fontsize=25)
plt.text(-2, 1, "LH", fontsize=25)
plt.text(-1, -1, "LL", fontsize=25)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14,12))
lisa_cluster(lisa, gdf, p=0.05, ax=ax)
plt.show()

Let's compare the different p-values.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

ax1, ax2 = axs

# regular count map on the left
lisa_cluster(lisa, gdf, p=0.05, ax=ax1)

ax1.axis("off")
ax1.set_title("P-value: 0.05")

# spatial lag map on the right
lisa_cluster(lisa, gdf, p=0.01, ax=ax2)
ax2.axis("off")
ax2.set_title("P-value: 0.01")

plt.show()

Now, let's create an interactive version of the LISA map.

In [ ]:
# original value list
lisa.y[:5]

In [ ]:
# quadrant list
lisa.q[:5]

In [ ]:
# p sim list
lisa.p_sim[:5]

In [ ]:
# add quadrant numbers to the dataframe
gdf['q'] = lisa.q.tolist()

In [ ]:
# add individual p-values to the dataframe
gdf['p_sim'] = lisa.p_sim.tolist()

In [ ]:
gdf.head()

In [ ]:
# Let's create a hotspot map
# identify just the hotspot geographies
hot_spots = gdf[(gdf.p_sim < 0.05) & (gdf.q == 1)]

In [ ]:
hot_spots.shape

In [ ]:
hot_spots.plot(figsize=(12,12),color='blue',legend=True,categorical=True)

In [ ]:
# interactive version needs to be in WGS84
hot_spots = hot_spots.to_crs('EPSG:4326')

In [ ]:
# what's the centroid?
minx, miny, maxx, maxy = hot_spots.geometry.total_bounds
center_lat_hot_spots = (maxy-miny)/2+miny
center_lon_hot_spots = (maxx-minx)/2+minx

In [ ]:
fig = px.choropleth_mapbox(hot_spots, 
                     geojson=hot_spots.geometry, 
                     locations=hot_spots.index, 
                     mapbox_style="satellite-streets",
                     center = {"lat": center_lat_hot_spots, "lon": center_lon_hot_spots},
                     zoom=9, 
                     opacity=0.6,
                     color='crimes_per_1000_lag',
                     color_continuous_scale='Blues',
                     color_continuous_midpoint =median,
                     range_color =(0,median*2),
                     hover_data=['crimes_count','crimes_per_1000','crimes_per_1000_lag'],
                    width=1200,
                     height=800,
                           labels={
                             'crimes_per_1000_lag':'Crimes Committed in Los Angeles per 1000 (Spatial Lag)',
                             'crimes_per_1000':'Crimes Committed in Los Angeles per 1000',
                     })
fig.update_traces(marker_line_width=1, marker_line_color='white')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Group Contributions
Donna conducted a spatial autocorrelation anaylsis using the Crime data for Los Angeles from lacity.org. Additionally, she included a global Moran's I statistic as well as a local spatial autocorrelation with a Moran's plot, indicating a p-value and a scatterplot with HH, HL, LH, and LL values.